<a href="https://colab.research.google.com/github/alvinaddaxis/Twitter-set/blob/main/Neo4jTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neo4j

In [7]:
import pandas as pd
import random
import string
from neo4j import GraphDatabase


In [16]:
# ceate random contents for nodes and relationships
ids=[]
names=[]
for i in range(100):
  ids.append(i) # generate 100 ids
for j in range(100):
  rand_str=''.join(random.choice(string.ascii_letters+string.digits) for _ in range(5))
  names.append(rand_str) # generate 100 names

start=[]
end=[]
prob=[]
for i in range(3):
  for j in range(100):
    start.append(j) # generate 300 start ids, involoving all ids
for i in range(300):
  end.append(random.randint(0,99)) #generate 300 end ids
  prob.append(round(random.uniform(0.1,1),2)) # generate 300 probabilities

random.shuffle(start)

100

In [ ]:
print(start)

[90, 49, 13, 36, 62, 43, 68, 55, 6, 96, 60, 11, 78, 33, 34, 85, 97, 54, 60, 68, 1, 81, 34, 26, 33, 4, 87, 50, 13, 91, 4, 1, 43, 58, 89, 89, 92, 78, 84, 62, 40, 2, 14, 9, 81, 25, 17, 80, 75, 38, 30, 6, 47, 5, 28, 45, 79, 36, 87, 84, 64, 84, 8, 23, 86, 74, 63, 59, 20, 46, 50, 7, 80, 69, 67, 43, 94, 39, 29, 48, 24, 79, 98, 17, 3, 41, 45, 91, 83, 16, 73, 30, 21, 10, 37, 36, 78, 99, 12, 39, 88, 66, 85, 25, 30, 76, 26, 92, 99, 42, 51, 71, 66, 20, 69, 59, 53, 96, 60, 26, 51, 50, 77, 91, 16, 46, 21, 73, 47, 40, 35, 15, 47, 93, 27, 15, 2, 89, 18, 31, 85, 74, 1, 0, 5, 97, 41, 65, 2, 22, 18, 16, 72, 46, 56, 22, 23, 72, 19, 72, 22, 29, 3, 71, 66, 95, 90, 54, 12, 13, 98, 97, 8, 59, 0, 56, 94, 95, 23, 88, 45, 52, 73, 49, 3, 48, 35, 51, 18, 57, 31, 19, 7, 64, 98, 63, 80, 10, 65, 67, 87, 24, 55, 11, 28, 62, 42, 69, 82, 6, 54, 94, 34, 49, 32, 64, 5, 14, 24, 52, 12, 67, 93, 70, 70, 37, 75, 77, 63, 40, 0, 29, 65, 81, 35, 83, 44, 55, 82, 25, 4, 61, 38, 58, 74, 8, 61, 70, 9, 52, 93, 68, 10, 15, 20, 79, 92,

In [21]:
# create two tables of nodes and relationships
df_node=pd.DataFrame({"id":ids,"name":names})
df_rel=pd.DataFrame({"from_id":start,"end_id":end,"probability":prob,"relation":"PROPAGATED"})


In [23]:
df_node.to_csv("prob_nodes.csv",index=False,sep=',')
df_rel.to_csv("prob_rel.csv",index=False,sep=',')


In [48]:

def execute_cql(cql_commands):
  # Neo4j Database Credentials
  uri="neo4j+s://ff7401b5.databases.neo4j.io:7687"
  userName="neo4j"
  password="6UXp56jDHcaZ2bCtRt89d5l1fyEJiYQ-OEmM9EpX_90"
  # Connect to the neo4j database server
  graphDB_Driver = GraphDatabase.driver(uri, auth=(userName,password))
  session = graphDB_Driver.session()
  for i in cql_commands:
    session.run(i)


In [ ]:
# cql operations

# to create Account nodes
creation_node_commands=[]
for i in ids:
  # single creation command
  command="MERGE (a:Account{id: toInteger(" + str(ids[i]) + "),name: '" + names[i] + "'})"
  creation_node_commands.append(command)

creation_node_commands

In [55]:
execute_cql(creation_node_commands)

In [ ]:
# to create relationships
creation_rel_commands=[]
for i in range(len(start)):
  # single creation command
  command="MATCH (start:Account{id: toInteger(" + str(start[i]) + ")}) MATCH (end:Account{id: toInteger(" + str(end[i]) + ")}) MERGE (start)-[:PROPAGATED{probability:toFloat("+str(prob[i])+")}]->(end)"
  creation_rel_commands.append(command)

creation_rel_commands

In [58]:
execute_cql(creation_rel_commands)

In [61]:
# test cql creation command
uri = "neo4j+s://ff7401b5.databases.neo4j.io:7687"
data_base_connection = GraphDatabase.driver(uri, auth=("neo4j", "6UXp56jDHcaZ2bCtRt89d5l1fyEJiYQ-OEmM9EpX_90"))
session = data_base_connection.session()
session.run("""CREATE (a:Account {id: toInteger(1111)})""")

In [62]:
# test cql updating command
session.run("MATCH (a:Account {id:1111}) SET a.name='Test'")

In [63]:
# test cql deleting command
session.run("MATCH (a:Account {id:1111}) DELETE a")

In [ ]:
nodes=session.run("MATCH (a:Account) RETURN a")
for node in nodes:
  print(node)